In [2]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier

LOCAL = './data/tanzania/'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Tree-Ensembles/master/data/tanzania/'
source = LOCAL

train = pd.merge(pd.read_csv(source + 'train_features.csv'), pd.read_csv(source + 'train_labels.csv'))

test = pd.read_csv(source + 'test_features.csv')
sample_submission = pd.read_csv(source + 'sample_submission.csv')

train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['status_group'], random_state=42)


def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()

    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    X['years'] = X['year_recorded'] - X['construction_year']    
    
    X = X.drop(columns=['recorded_by', 'id'])
    
    duplicate_columns = ['quantity_group']
    X = X.drop(columns=duplicate_columns)
    
    X['latitude'] = X['latitude'].replace(-2e-08, np.nan)
    
    cols_with_zeros = ['construction_year', 'longitude', 'latitude', 'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [3]:
target = 'status_group';

train_features = train.drop(columns = target);
numeric_features = train_features.select_dtypes(include='number').columns.tolist();
cardinality = train_features.select_dtypes(exclude='number').nunique();
categorical_features = cardinality[cardinality <= 50].index.tolist();
features = numeric_features+categorical_features;

In [40]:
from sklearn.ensemble import RandomForestClassifier;

#x_train = train[features];
y_train = train[target];

#x_val = val[features];
y_val = val[target];

#x_test = test[features];
x_train = train.drop(columns=target);
x_val = val.drop(columns=target);
x_test = test;

pipeline = make_pipeline(
ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=107, random_state=20, n_jobs=-1)
);

pipeline.fit(x_train, y_train);

print(pipeline.score(x_val, y_val))
y_pred = pipeline.predict(x_test)

submission = sample_submission.copy();
submission['status_group'] = y_pred;
submission.to_csv('Ashwin-Raghav-Swamy-Kaggle', index=False);

0.8125420875420876


n_estimators=107, random_state=20 = 0.0.8125420875420876